In [ ]:
## ⚠️ Important Note (Read Before Running Anything)
'''
This notebook exists **only** to clean and normalize the original Kaggle CSV
so it can be safely used for **model training and synthetic data generation**.

Why this matters:
- SDV / CTGAN expects a **clean, stable CSV**
- Broken encodings, bad dates, and messy column names will cause training to fail
- This cleanup is mandatory before feeding data to any model

Scope:
- The output of this notebook is a **model-ready CSV**
- It is not intended for analysis or dashboards
- All blocks are intentionally separate and may be run independently

If you skip this step and jump straight to training, the model will break.
That's not a bug. That's on you.
'''

In [ ]:
## Initial Text Normalization & Column Pruning (Raw Kaggle Cleanup)
'''
This block performs aggressive text normalization on the original Kaggle
DataCo Supply Chain dataset to eliminate encoding issues and unusable columns.

Purpose:
- Remove accented and corrupted Unicode characters
- Normalize all text fields to plain ASCII
- Drop unused, sensitive, or redundant columns
- Produce a stable, model-safe CSV for downstream processing

Notes:
- This is a one-time cleanup step for a single raw file
- Blocks in this notebook are intentionally independent
- File paths and names can be adjusted as needed
'''

In [ ]:
import pandas as pd
import unicodedata

def clean_text(text):
    """
    Normalize accented or special Unicode characters into plain ASCII.
    Example: 'São Paulo' -> 'Sao Paulo'
    """
    if pd.isna(text):
        return ""
    
    text = str(text)
    
    # Decompose Unicode characters (e.g., é -> e + accent)
    # Retain only base characters
    return ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    )

# ------------------------------------------------------------------------------
# 1. LOAD RAW DATA
# ------------------------------------------------------------------------------
print("Loading raw dataset for text normalization...")

# 'latin-1' is more forgiving for this dataset's encoding issues
df = pd.read_csv(
    'DataCoSupplyChainDataset.csv',
    encoding='latin-1'
)

# ------------------------------------------------------------------------------
# 2. NORMALIZE COLUMN NAMES
# ------------------------------------------------------------------------------
# Strip accidental whitespace from column headers
df.columns = df.columns.str.strip()

# ------------------------------------------------------------------------------
# 3. APPLY TEXT NORMALIZATION ACROSS ALL TEXT COLUMNS
# ------------------------------------------------------------------------------
print("Normalizing special characters across text columns...")

text_cols = df.select_dtypes(include=['object']).columns

for col in text_cols:
    df[col] = df[col].apply(clean_text)

# ------------------------------------------------------------------------------
# 4. DROP UNUSED / SENSITIVE / REDUNDANT COLUMNS
# ------------------------------------------------------------------------------
cols_to_drop = [
    'Customer Email', 'Customer Fname', 'Customer Lname', 'Customer Password',
    'Customer Street', 'Customer Zipcode', 'Order Id', 'Order Item Id',
    'Customer Id', 'Order Customer Id', 'Product Card Id',
    'Order Item Cardprod Id', 'Category Id', 'Department Id',
    'Product Category Id', 'Product Description', 'Product Image',
    'Latitude', 'Longitude', 'Benefit per order', 'Sales per customer',
    'Delivery Status', 'Late_delivery_risk', 'Customer City', 'Order City',
    'Order Item Discount', 'Sales', 'Order Item Total', 'Order Profit Per Order',
    'Order Zipcode', 'Product Price', 'Product Status',
    'shipping date (DateOrders)'
]

# Drop only columns that actually exist
existing_cols = [c for c in cols_to_drop if c in df.columns]
df_clean = df.drop(columns=existing_cols)

print(f"Dropped {len(existing_cols)} columns.")

# ------------------------------------------------------------------------------
# 5. EXPORT CLEANED DATA
# ------------------------------------------------------------------------------
# 'utf-8-sig' ensures Excel correctly interprets UTF-8 encoding
output_name = 'DataCo_Cleaned_Plain_English.csv'
df_clean.to_csv(
    output_name,
    index=False,
    encoding='utf-8-sig'
)

print(f"Cleanup complete. Output saved as: {output_name}")

In [ ]:
## Date Parsing & Temporal Repair (Raw Kaggle Fix)
'''
This block repairs corrupted and inconsistently formatted order date fields
from the original Kaggle dataset.

Purpose:
- Parse multiple inconsistent datetime formats
- Extract year, month, and day as atomic columns
- Correct malformed years (e.g., 0018 → 2018)
- Recompute day-of-week after date correction

Notes:
- This is a best-effort cleanup for a single raw file
- Imperfect rows are tolerated and corrected logically
- This step exists solely to stabilize downstream analysis
'''

In [ ]:
import polars as pl
import os

# ------------------------------------------------------------------------------
# 1. FILE SETUP
# ------------------------------------------------------------------------------
folder_path = r"D:\Data Lake\very_raw"
file_name = "supply_chain_sample_new_2.csv"
full_path = os.path.join(folder_path, file_name)

df = pl.read_csv(full_path)

# ------------------------------------------------------------------------------
# 2. CLEAN & PARSE DATE STRINGS
# ------------------------------------------------------------------------------
# Normalize separators and spacing before parsing
df = df.with_columns(
    date_str_clean=(
        pl.col("order date (DateOrders)")
        .str.replace_all("/", "-")
        .str.replace_all("  ", " ")
        .str.strip_chars()
    )
)

# Attempt multiple datetime formats (best-effort parsing)
df_final = df.with_columns(
    parsed_date=pl.coalesce(
        pl.col("date_str_clean").str.to_datetime("%m-%d-%Y %H:%M", strict=False),
        pl.col("date_str_clean").str.to_datetime("%m-%d-%Y %I:%M:%S %p", strict=False),
        pl.col("date_str_clean").str.to_datetime("%m-%d-%y %H:%M", strict=False)
    )
).with_columns(
    Order_Year=pl.col("parsed_date").dt.year(),
    Order_Month=pl.col("parsed_date").dt.month(),
    Order_Day=pl.col("parsed_date").dt.day()
)

# ------------------------------------------------------------------------------
# 3. TEMPORAL CORRECTION (YEAR FIX)
# ------------------------------------------------------------------------------
# Fix malformed years (e.g., 18 -> 2018)
df_final = df_final.with_columns(
    Order_Year=pl.when(pl.col("Order_Year") < 1900)
                  .then(pl.col("Order_Year") + 2000)
                  .otherwise(pl.col("Order_Year"))
)

# ------------------------------------------------------------------------------
# 4. RECOMPUTE DAY OF WEEK
# ------------------------------------------------------------------------------
# Recalculate weekday after correcting the year
df_final = df_final.with_columns(
    Order_DayOfWeek=pl.date(
        pl.col("Order_Year"),
        pl.col("Order_Month"),
        pl.col("Order_Day")
    ).dt.weekday()
).drop(["date_str_clean", "parsed_date"])

# ------------------------------------------------------------------------------
# 5. VALIDATION CHECK
# ------------------------------------------------------------------------------
print("Validating year correction...")

ancient_check = df_final.filter(pl.col("Order_Year") < 1900)

if ancient_check.height == 0:
    print("All years successfully corrected.")
    print(
        df_final.select(
            ["order date (DateOrders)", "Order_Year", "Order_DayOfWeek"]
        ).head(10)
    )
else:
    print(f"Found {ancient_check.height} rows with invalid years.")

# ------------------------------------------------------------------------------
# 6. EXPORT CLEANED DATA
# ------------------------------------------------------------------------------
output_filename = "Cleaned_Data_With_Dates.csv"
output_path = os.path.join(folder_path, output_filename)

print(f"Saving cleaned dataset to: {output_path}")

df_final.write_csv(
    output_path,
    separator=","
)

print("Date cleanup completed.")

In [ ]:
## Column Name Standardization (Final Schema Hygiene)
'''
This block standardizes column names after all structural and date fixes
have been applied.

Purpose:
- Enforce consistent snake_case naming
- Remove spaces, brackets, and malformed characters
- Prepare columns for downstream SQL, Polars, and BI compatibility

Notes:
- This step is intentionally isolated
- Naming logic is explicit to avoid silent schema drift
- Output is the final, model-ready dataset
'''

In [ ]:
import polars as pl
import os

# ------------------------------------------------------------------------------
# COLUMN NAME STANDARDIZATION
# ------------------------------------------------------------------------------
folder_path = r"D:\Data Lake\very_raw"
file_name = "Cleaned_Data_With_Dates.csv"
full_path = os.path.join(folder_path, file_name)

df2 = pl.read_csv(full_path)

new_columns = []

for col in df2.columns:
    # Step 1: Strip leading/trailing whitespace
    clean_name = col.strip()

    # Step 2: Normalize casing
    clean_name = clean_name.lower()

    # Step 3: Replace problematic characters with underscores
    clean_name = (
        clean_name
        .replace(" ", "_")
        .replace("(", "_")
        .replace(")", "_")
        # Extend replacements here if additional symbols appear
    )

    # Step 4: Collapse multiple underscores
    while "__" in clean_name:
        clean_name = clean_name.replace("__", "_")

    # Step 5: Trim underscores from edges
    clean_name = clean_name.strip("_")

    new_columns.append(clean_name)

# Apply cleaned column names
df2.columns = new_columns

# Verify results
print("Column name standardization complete.")
print(df2.columns)

# ------------------------------------------------------------------------------
# EXPORT FINAL CLEAN DATASET
# ------------------------------------------------------------------------------
output_filename = "fixed_columns_final.csv"
output_path = os.path.join(folder_path, output_filename)

df2.write_csv(output_path, separator=",")

print(f"Final dataset saved to: {output_path}")